In [1]:
import json
import pandas as pd
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer

In [2]:
with open('document_with_ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [3]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

### Indexing

In [4]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)
    

  0%|          | 0/948 [00:00<?, ?it/s]

In [5]:
es_client = Elasticsearch("http://localhost:9200")

index_settings = {
    "settings": {
        "number_of_shards" : 1,
        "number_of_replicas" : 0
    },
    "mappings" : {
        "properties" : {
            "text" : {"type" : "text"},
            "section" : {"type" : "text"},
            "question" : {"type" : "text"},
            "course" : {"type" : "keyword"},
            "id" : {"type" : "keyword"},
            "question_vector" : {
                "type" : "dense_vector",
                "dims" : 384,
                "index" : True,
                "similarity": "cosine"
            },
            "text_vector" : {
                "type" : "dense_vector",
                "dims" : 384,
                "index" : True,
                "similarity": "cosine"
            },
            "question_text_vector" : {
                "type" : "dense_vector",
                "dims" : 384,
                "index" : True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course_questions"
es_client.indices.delete(index = index_name, ignore_unavailable=True)
es_client.indices.create(index = index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_questions'})

In [6]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

#### Retrieval stage